# Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import holoviews as hv

from six.moves import xrange
from scipy.spatial import distance_matrix
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import routing_enums_pb2, pywrapcp
# from ortools.graph.python import pywrapgraph

import time
import pprint
import math
from collections import Counter
from glob import glob
from IPython.display import Image, display
from IPython.core.display import HTML

pd.options.display.max_rows = 999
pd.options.display.width = 300
pd.options.display.max_columns = 500
start = time.time()

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Route Optimization\\OR-Tools'

# Data Wrangling

In [4]:
def list_lines(file_name):
    '''Returns a list of integer lists.'''
    with open(file_name) as file:
        lines = file.read().splitlines()
    line_list = [[int(n) for n in ll.split()] for ll in lines]
    return line_list

def set_params(line_list):
    top_line = line_list[0]
    params = {
        'DRONE_COUNT': top_line[2],
        'WT_CAP': top_line[4],
        'END_TIME': top_line[3]
    }
    return params

def find_wh_lines(line_list):
    '''Provides the dividing line between the warehouse and order sections in the line list.'''
    wh_count = line_list[3][0]
    wh_endline = (wh_count * 2) + 4
    return wh_endline

def get_weights(line_list):
    weights = np.arrayt(line_list[2])
    return weights.astype(np.int16)

def get_inventories(line_list):
    '''Returns a 2D array of P products by W warehouses.'''
    wh_endline = find_wh_lines(line_list)
    invs = line_list[5:wh_endline + 1:2]
    supply  = np.array(invs).transpose()
    return supply.astype(np.int16)

def get_orders(line_list):
    '''Returns a 2D array of P products by C orders.'''
    wh_endline = find_wh_lines(line_list)
    demand = np.zeros((line_list[1][0], line_list[wh_endline][0]),
                        dtype = np.int16)
    orders = line_list[wh_endline+3::3]
    for i, ord in enumerate(orders):
        for prod in ord:
            demand[prod, i] += i
    return demand.astype(np.int16)

def get_locs(line_list):
    wh_endline = find_wh_lines(line_list)
    wh_locs = np.array(line_list[4:wh_endline:2])
    cust_locs = np.array(line_list[wh_endline+1::3])
    return wh_locs.astype(np.int16), cust_locs.astype(np.int16)